In [47]:
import pandas as pd

# Load the CSV into a pandas DataFrame
df = pd.read_csv('jyllands_posten_data.csv')

# Display the first few rows to get an overview
df


,Term,Date,Headline,Category
0,havstigninger,2011-01-02,Husejere vil kende klima-risiko,Bolig
1,havstigninger,2011-01-03,Klar besked om klimarisiko,JP Erhverv
2,klimaforandringer,2011-01-03,295.000 dræbt af naturen i 2010,International
3,CO2-udledning,2011-01-04,Danmarks grønne industri visner,JP Erhverv
4,klimaforandringer,2011-01-04,Kinas ørkner vil bestå i 300 år,Viden
...,...,...,...,...
14224,CO2-mål,2023-08-13,"Kina er i gang med et megaprojekt, og det bliv...",Asien
14225,CO2-udledning,2023-08-13,Ambitiøse klimamål er ikke nok – der er også b...,Debatindlæg
14226,klimakrisen,2023-08-14,Ophæv forbuddet mod atomkraft,Debatindlæg
14227,klimakrisen,2023-08-14,"Her bestemmer børnene selv, hvornår de møder",JP Aarhus


In [48]:
# Convert the "Date" column to standard datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')

# Extract the "yyyy-mm" to a new column that matches weather data
df['month'] = df['Date'].dt.strftime('%Y-%m')

# Display the last few rows to confirm the changes
df

,Term,Date,Headline,Category,month
0,havstigninger,2011-01-02,Husejere vil kende klima-risiko,Bolig,2011-01
1,havstigninger,2011-01-03,Klar besked om klimarisiko,JP Erhverv,2011-01
2,klimaforandringer,2011-01-03,295.000 dræbt af naturen i 2010,International,2011-01
3,CO2-udledning,2011-01-04,Danmarks grønne industri visner,JP Erhverv,2011-01
4,klimaforandringer,2011-01-04,Kinas ørkner vil bestå i 300 år,Viden,2011-01
...,...,...,...,...,...
14224,CO2-mål,2023-08-13,"Kina er i gang med et megaprojekt, og det bliv...",Asien,2023-08
14225,CO2-udledning,2023-08-13,Ambitiøse klimamål er ikke nok – der er også b...,Debatindlæg,2023-08
14226,klimakrisen,2023-08-14,Ophæv forbuddet mod atomkraft,Debatindlæg,2023-08
14227,klimakrisen,2023-08-14,"Her bestemmer børnene selv, hvornår de møder",JP Aarhus,2023-08


In [49]:
# Identify duplicate rows based on all columns except "Term"
duplicates = df[df.duplicated(subset=['Date', 'Headline', 'Category'], keep=False)]

# Display the duplicate rows
duplicates.sort_values(by=['Date', 'Headline', 'Category'])

# Count the number of duplicated rows based on 'Date', 'Headline', and 'Category'
num_duplicates = df.duplicated(subset=['Date', 'Headline', 'Category']).sum()

print(f"Number of duplicated rows: {num_duplicates}")

# Drop duplicated rows based on 'Date', 'Headline', and 'Category'
df = df.drop_duplicates(subset=['Date', 'Headline', 'Category'], keep='first')

# Print the comment with the number of duplicates removed
print(f"{num_duplicates} rows have been removed.")

# Count the number of articles per month and merge with the main DataFrame
df = df.merge(df.groupby('month').size().reset_index(name='article count'), on='month')

Number of duplicated rows: 4341
4341 rows have been removed.


In [50]:
# Save the half-cleaned data to a new file called "news_testing.csv" for correlation testing
df.to_csv('News_testing.csv', index=False)
print("Data saved to 'News_testing.csv'")
df

Data saved to 'News_testing.csv'


,Term,Date,Headline,Category,month,article count
0,havstigninger,2011-01-02,Husejere vil kende klima-risiko,Bolig,2011-01,49
1,havstigninger,2011-01-03,Klar besked om klimarisiko,JP Erhverv,2011-01,49
2,klimaforandringer,2011-01-03,295.000 dræbt af naturen i 2010,International,2011-01,49
3,CO2-udledning,2011-01-04,Danmarks grønne industri visner,JP Erhverv,2011-01,49
4,klimaforandringer,2011-01-04,Kinas ørkner vil bestå i 300 år,Viden,2011-01,49
...,...,...,...,...,...,...
9883,CO2-mål,2023-08-13,"Kina er i gang med et megaprojekt, og det bliv...",Asien,2023-08,36
9884,CO2-udledning,2023-08-13,Ambitiøse klimamål er ikke nok – der er også b...,Debatindlæg,2023-08,36
9885,klimakrisen,2023-08-14,Ophæv forbuddet mod atomkraft,Debatindlæg,2023-08,36
9886,klimakrisen,2023-08-14,"Her bestemmer børnene selv, hvornår de møder",JP Aarhus,2023-08,36


In [51]:
# Count the number of articles per month
monthly_article_counts = df.groupby('month').size().reset_index(name='article count')

# Merge the original DataFrame with the monthly article counts
df = df.merge(monthly_article_counts, on='month', how='left')

# Drop the original 'article count' column (if it exists) to avoid duplication
if 'article count_x' in df.columns:
    df = df.drop(columns=['article count_x'])
if 'article count_y' in df.columns:
    df.rename(columns={'article count_y': 'article count'}, inplace=True)

# Display the first few rows to confirm
df.head()


,Term,Date,Headline,Category,month,article count
0,havstigninger,2011-01-02,Husejere vil kende klima-risiko,Bolig,2011-01,49
1,havstigninger,2011-01-03,Klar besked om klimarisiko,JP Erhverv,2011-01,49
2,klimaforandringer,2011-01-03,295.000 dræbt af naturen i 2010,International,2011-01,49
3,CO2-udledning,2011-01-04,Danmarks grønne industri visner,JP Erhverv,2011-01,49
4,klimaforandringer,2011-01-04,Kinas ørkner vil bestå i 300 år,Viden,2011-01,49


In [52]:
# Save the half-cleaned data to a new file called "news_testing.csv" for correlation testing
df.to_csv('News_testing.csv', index=False)
print("Data saved to 'News_testing.csv'")

Data saved to 'News_testing.csv'


In [53]:
# Remove unnecessary columns to retain only the "month" and "Articles this month" columns
df = df.drop(columns=['Term', 'Date', 'Headline', 'Category']).drop_duplicates()

df

,month,article count
0,2011-01,49
49,2011-02,53
102,2011-03,37
139,2011-04,44
183,2011-05,52
...,...,...
9565,2023-04,68
9633,2023-05,82
9715,2023-06,71
9786,2023-07,66


In [54]:
# Save the cleaned data to a new file called "News.csv" 
df.to_csv('News_data.csv', index=False)

print("Data saved to 'News_data.csv'")

Data saved to 'News_data.csv'
